## Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

In [2]:
#training data
#inputs -> temp, humidity, rainfall
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 76, 90]], dtype='float32')

#targets -> apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [3]:
#inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

#### Model

In [4]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.8985, -1.1727, -0.4134],
        [-0.6080, -0.0193, -0.7794]], requires_grad=True)
tensor([-0.3192, -0.5804], requires_grad=True)


In [5]:
#model
def model(x):
    return x @ w.t() + b        #@ -> matrix multiplication in pytorch; t-> transpose

In [6]:
#predictions
preds = model(inputs)
print(preds)

tensor([[  41.9262,  -79.7778],
        [  42.7916, -107.4966],
        [ -16.2639, -101.2776],
        [ 127.6066,  -92.2697],
        [   4.3479, -114.1533]], grad_fn=<AddBackward0>)


In [7]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#loss function (mse)
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(preds, targets)
loss.sqrt()

tensor(152.5453, grad_fn=<SqrtBackward0>)

In [10]:
loss.backward()

In [11]:
print(w)
print(w.grad)

tensor([[ 1.8985, -1.1727, -0.4134],
        [-0.6080, -0.0193, -0.7794]], requires_grad=True)
tensor([[ -2461.4883,  -5077.4263,  -3173.4136],
        [-15912.4443, -16610.8535, -11827.8232]])


In [12]:
#resetting gradients back to 0 to prevent adding up
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


##### Gradient Descent

In [13]:
preds = model(inputs)
preds

tensor([[  41.9262,  -79.7778],
        [  42.7916, -107.4966],
        [ -16.2639, -101.2776],
        [ 127.6066,  -92.2697],
        [   4.3479, -114.1533]], grad_fn=<AddBackward0>)

In [15]:
loss = mse(preds, targets)
loss

tensor(23270.0664, grad_fn=<DivBackward0>)

In [16]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -2461.4883,  -5077.4263,  -3173.4136],
        [-15912.4443, -16610.8535, -11827.8232]])
tensor([ -36.1183, -190.9950])


In [18]:
#adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [19]:
#new weights and biases
print(w)
print(b)

tensor([[ 1.9477, -1.0711, -0.3499],
        [-0.2898,  0.3129, -0.5429]], requires_grad=True)
tensor([-0.3188, -0.5785], requires_grad=True)


In [21]:
#re-calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(11550.7188, grad_fn=<DivBackward0>)